In [1]:
import sys
sys.path.append('../code')
sys.path.append('../')

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import neo
import pandas as pd
import scipy
from externals.SpectralEvents import spectralevents as se
import moviepy.editor

import mocap_functions, spike_train_functions, lfp_functions
from functools import partial
import pickle
import seaborn as sns


In [2]:
experiment_dict = sio.loadmat('../data/SPK20220308/task_data/eventsCB1_corrected2.mat')
ev_ex = experiment_dict['eventsCB1']

tgtON = ev_ex['tgtON_C'][0][0][0]
gocON = ev_ex['gocON_C'][0][0][0]

endhold = ev_ex['endhold_C'][0][0][0]

position = ev_ex['POSITION_C'][0][0][0]
layout = ev_ex['LAYOUT_C'][0][0][0]

gocON_all = ev_ex['gocON'][0][0][0]
tgtON_all = ev_ex['tgtON'][0][0][0]

go_correct_mask = np.in1d(gocON_all, gocON)
go_error_mask = ~go_correct_mask

cam_start = 14.851
downsample_rate = 30
samp_freq = 1000

#Use neo module to load blackrock files
fpath = '../data/SPK20220308/task_data/SPKRH20220308_CB1_quiver4toyPKPK4Rotation_delay_001'
lfp_data_raw, lfp_times_raw = lfp_functions.load_ns6_analog(fpath, downsample_rate, from_ns6=False, save=False, channel_step=2)

In [4]:
st_offset = -1 # seconds
max_time = 2 # seconds
max_samples = int(max_time * samp_freq)
channel_idx = 50

lfp_event_ts = gocON


lfp_data_list = list()
for event_time in lfp_event_ts:
    tstart = event_time + st_offset
    start_idx = np.argmin(np.abs(lfp_times_raw - tstart))
    stop_idx = start_idx + max_samples

    lfp_data_list.append(lfp_data_raw[channel_idx, start_idx:stop_idx])

lfp_data = np.stack(lfp_data_list)

power_samples = (0, 1000) # range to calculate band power on
power_list = np.array([lfp_functions.bandpower(
    lfp_data[idx, power_samples[0]:power_samples[1]], fs=samp_freq, fmin=13, fmax=29) for 
    idx in range(lfp_data.shape[0])])

power_sort = np.argsort(power_list)[::-1]


In [5]:
estart_times = tgtON[power_sort]
estop_times = endhold[power_sort]

for cam_idx in range(5,6):
    clip = moviepy.editor.VideoFileClip(f'../data/SPK20220308/videos/SpikeCam{cam_idx}_03-08-1557.mp4')

    clip_list = list()
    for trial_idx, (tstart, tstop) in enumerate(zip(estart_times - 1, estop_times)):
        trial_clip = clip.subclip(tstart - cam_start, tstop - cam_start)
        clip_list.append(trial_clip)


    # concatenating both the clips
    final = moviepy.editor.concatenate_videoclips(clip_list)

    #writing the video into a file / saving the combined video
    final.write_videofile(f'../data/SPK20220308/videos_beta_power/SpikeCam{cam_idx}_03-08-1557_beta_sort.mp4', fps=40)


Moviepy - Building video ../data/SPK20220308/videos_beta_power/SpikeCam6_03-08-1557_beta_sort.mp4.
Moviepy - Writing video ../data/SPK20220308/videos_beta_power/SpikeCam6_03-08-1557_beta_sort.mp4



Moviepy - Done !
Moviepy - video ready ../data/SPK20220308/videos_beta_power/SpikeCam6_03-08-1557_beta_sort.mp4


In [7]:
# Stack clips into array
clip_list = list()
for cam_idx in [1,6]:
    clip = moviepy.editor.VideoFileClip(f'../data/SPK20220308/videos_beta_power/SpikeCam{cam_idx}_03-08-1557_beta_sort.mp4')
    clip_list.append(clip)
clips_array = [[clip_list[0], clip_list[1]]]

In [8]:
final_clip = moviepy.editor.clips_array(clips_array)
final_clip.write_videofile("Spike_03-08-1557_beta_sort_combined.mp4")

Moviepy - Building video Spike_03-08-1557_beta_sort_combined.mp4.
Moviepy - Writing video Spike_03-08-1557_beta_sort_combined.mp4



Moviepy - Done !
Moviepy - video ready Spike_03-08-1557_beta_sort_combined.mp4
